# CREASE for Vesicle Analysis

## Introduction

**This tutorial has been created by Ziyu Ye, Zijie Wu, and Arthi Jayaraman to accompany their publication: Ye, Z.; Wu, Z; Jayaraman, A. Computational Reverse-Engineering Analysis for Scattering Experiments (CREASE) on Vesicles Assembled from Amphiphilic Macromolecular Solutions (under review)**

In general, Computational Reverse Engineering Analysis for Scattering Experiments (CREASE) is a two step approach that combines a genetic algorithm (GA) as the first step with molecular simulations based reconstruction as the second step to determine for a given input scattering profile the structural features of assembled structures in macromolecular solutions. Structural features range from aggregate dimensions to the chain- and monomer- level packing within the aggregate. This tutorial presents CREASE for vesicles. If you use CREASE in your work, we ask that you please cite one of the relevant papers below. If you use this vesicle code, please cite References 5. 


[1] Beltran-Villegas, D. J.; Wessels, M. G.; Lee, J. Y.; Song, Y.; Wooley, K. L.; Pochan, D. J.; Jayaraman, A. Computational Reverse-Engineering Analysis for Scattering Experiments on Amphiphilic Block Polymer Solutions. *J. Am. Chem. Soc.* **2019**, 141, 14916−14930.

[2] Wessels, M. G.; Jayaraman, A. Computational Reverse-Engineering Analysis of Scattering Experiments (CREASE) on Amphiphilic Block Polymer Solutions: Cylindrical and Fibrillar Assembly. *Macromolecules* **2021**, 54, 783-796.

[3] Wessels, M. G.; Jayaraman, A. Machine Learning Enhanced Computational Reverse Engineering Analysis for Scattering Experiments (CREASE) to Determine Structures in Amphiphilic Polymer Solutions. *ACS Polymers Au* **2021**, https://doi.org/10.1021/acspolymersau.1c00015.

[4] Heil, C.; Jayaraman, A. Computational reverse-engineering analysis for scattering experiments of assembled binary mixture of nanoparticles. *ACS Materials Au* **2021**, https://doi.org/10.1021/acsmaterialsau.1c00015

[5] Ye, Z.; Wu, Z; Jayaraman, A. Computational Reverse-Engineering Analysis for Scattering Experiments (CREASE) on Vesicles Assembled from Amphiphilic Macromolecular Solutions.

This notebook presents a tutorial on how the first step of CREASE, the GA step, works to determine the structural features of the assembled structure (vesicle in this tutorial) found in macromolecular solutions from small angle scattering intensity profiles, I(q)s. The GA codes takes as input an I(q) and finds the structural dimensions of the vesicle whose computed scattering, 𝐼𝑐𝑜𝑚𝑝(q), best matches the input I(q). The GA code presented here is applicable to a vesicle structure with an empty (solvent filled) core and three layers (two solvophilic layers and one solvophobic layer). But it can be adapted to other shapes and geometries by modifying sections of the code that will be highlighted as such.

The GA code for vesicles is able to determine:
1. Vesicle dimensions: the core radius ($R_{core}$), the inner solvophilic A layer thickness ($t_{Ain}$), the middle solvophobic B layer thickness ($t_{B}$), and the outer solvophilic A layer thickness ($t_{Aout}$) 
2. How the solvophilic scatterers are split between inner ($s_{Ain}$) and outer ($1-s_{Ain}$) layers.
3. Dispersity (or polydispersity) in vesicle size (here implemented in the core radius dimension ($\sigma_{R}$) during I(q) calculation).

<img src="CREASE_vesicles_code_structure_dimensions.png" width=300 height=300 />

A basic understanding of how a GA works and the associated technical terms (e.g. gene, generation, selection, fitness, crossover, mutation) would be helpful for following along to this tutorial. The user can find a brief introduction to GA from any of the five references listed above, especially in the original refences [1]. The [the wikipedia page of genetic algorithm](https://en.wikipedia.org/wiki/Genetic_algorithm) also offers a general overview of the GA method.

Below is the general structure showing the flow of the GA code presented in this package:

<img src="tutorial_images/Slide2.PNG" width=600 height=600 />

## Setup python libraries

In [ ]:
import crease_ga as cga
from inspect import getsource

### Model

Current "location" in the code flow diagram marked in red:

<img src="tutorial_images/Slide3.PNG" width=600 height=600 />

Let's start with the initial arguments that the user needs to choose and input for their system. 

In the cell block below, the user needs to define the arguments that define the basic architecture of the genetic algorithm:
* number of individuals in a population ('pop_number')
* number of generations that GA runs for ('generations')
* number of binary digits used to represent each individual ('nloci')

The user chooses the above three agruments to run the GA. The choice of these three arguments will significantly affect the performance of the GA. We recommend that the user test out different combinations of these three parameters to ensure convergence to the right output. For the vesicle example, after multiple tests during our previous work, we recommend the following values for these arguments to produce reliable output for the input $I$(q):
* `pop_number = 80`
* `generations = 100`
* `nloci = 7`

Because the computational time needed to complete the GA run increases with increasing number of individuals and generations, for the purpose of completing this tutorial within reasonable time on a laptop or desktop, we use the following values:
* `pop_number = 5`
* `generations = 5`
* `nloci = 7`

You should not expect to see the 'best' 𝐼𝑐𝑜𝑚𝑝(q) match the input or target $I$(q) with these settings. With a larger number of individuals and generations, you can see how the GA predictions evolve over the generations to higher "fitness" structures [i.e. closer match to the input $I$(q)].

For each individual in the GA, we store information about structural features as genes in its genome. For each of the individual's genes, the information is encoded as a seven-digit (chosen by the user as $n_{loci}$, set in this tutorial as 7) binary sequence that corresponds to the value of structural dimensions evenly distributed between the maximum and minimum bounds set by the user (minimum bound will be 0000000, and maximum 1111111). The choice of the number of binary digits will determine the precision of the final optimized structural dimensions.

*You may ask at this point - why are we encoding the parameters in a binary fashion?*
That is a great question! Actually, whether to use binary or decimal encoding has always been a topic of debate in the field of genetic algorithm. Traditional GAs tend to use binary coding (referred to as 'binary-coded GA'), but there also have been numerous applications of GA to engineering problems using "decimal coding" (often referred to as 'real-coded GA'). We developed the algorithm following the binary coding route, but there is technically no reason why one cannot use real-coded GA for this problem. One can always rewrite our genetic operator algorithms if one wishes to use real-coded GA (more on this later)

In [ ]:
m = cga.Model(pop_number = 5,
              generations = 5,
              nloci = 7)

## load_shape

<img src="tutorial_images/Slide4.PNG" width=600 height=600 />

The second set of arguments passed to the GA is done through `load_shape`, and here, the user chooses the assembled morphology they are attempting to match. For this tutorial the shape is 'vesicle'.

**shape_params**

Regardless of the type of assembled shape (vesicles or other micelles), each shape requires the user to choose a set of shape descriptors that defines the basic morphology of the structure. In this example, we load in the `vesicle` shape, which requires the following 7 shape descriptors (specified as a list given in the following order with the `shape_params` argument):
* Number of scatterers per chain (${num}_{scatterers}$)
* Number of beads on chain ($N$)
* Number density of beads in B layer ($\rho_B$) [Angstrom$^{-3}$]
* Monomer contour length' ($l_{mono,b}$） [Angstrom]
* Monomer cminontour length' ($l_{mono,a}$）[Angstrom]
* Fraction of B type monomers in chain ($fb$)
* Number of replicates ($n_{LP}$)

For vesicles assembled from amphiphilic macromolecules with solvophilic A and solvophobic B blocks, two types of scatterers (A and B, respectively) are randomly placed within the designated vesicle layer boundaries (more details to be given later). The ratio of A and B scatterers matches the composition of the amphiphilic macromolecule ( 1−𝑓𝑏 ,  𝑓𝑏 ).  We note that the scatterers could be point scatterers or scatterers with a specified diameter, as is the case in this example (for reasons described in the reference [5]). Regardless of the choice, the total number of A and B scatterers is another choice that the user has to make to balance the accuracy of the output vesicle dimensions from GA and the computational time for the 𝐼𝑐𝑜𝑚𝑝(q) calculation (more details to be given later).

NOTE: The determination of 𝐼𝑐𝑜𝑚𝑝(q) in the GA only uses the scatterer placements and these placements do NOT include any connectivity to resemble macromolecules [i.e. no polymer chains are placed].

In this vesicle example, the user can enter the macromolecular information about the solution that was used to obtain the scattering profile to guide the choice of number of scatterers placed in the individual candidate structure. Here for this tutorial example, we use the molecular information for a diblock copolymer with the solvophilic block A and solvophobic block B forming the vesicle bilayer. The diblock chain has total number of  𝑁 monomer beads with  𝑓𝑏  fraction of the beads in the chain being solvophobic. The number density of the solvophobic monomor beads in the middle layer is given by $\rho_B$ and the diameters of the A an B beads (or "monomer contour length") are given by $l_{mono,a}$ and $l_{mono,b}$, respectively. The specific values used in this example are given below.

For this vesicle system example, we calculate each individual's 𝐼𝑐𝑜𝑚𝑝(q) using seven different replicates/configurations (`nLP`) of random scatterer placements to generate the averaged 𝐼𝑐𝑜𝑚𝑝(q) for that individual. Having multiple configurations of scatterer placements mitigates the chance that one configuration of scatterer placements biases the computed scattering intensity. One could also choose to replace the use of multiple (`nLP=7` in this case) placements of fewer scatterers with just one placement using a significantly higher total number of scatterers per unit volume (i.e., scatterer density) leading to a reduced bias compared to using fewer scatterers (or smaller scatterer density). We choose to use multiple different configurations of random scatterer placements in this case to incorporate the ability to capture dispersity in dimensions (more details will be found later). We also note that the calculation of 𝐼𝑐𝑜𝑚𝑝(q) scales quadratically with the number of scatterers used, providing another reason for multiple replicates.

For a more detailed discussion on the scatterer placement and density in the specific case of CREASE for vesicles, please see reference [5].

**Setting bounds for GA parameters**

The user also sets the maximum (`maxvalu`) and minimum (`minvalu`) bounds for each parameter (or gene) of the individual candidate structure. For the vesicle system example, the GA determines the following parameters:
* Core radius ($R_{core}$) 
* Three individual layer thicknesses in the bilayer consisting of the inner solvophilic A layer ($t_{Ain}$), middle solvophobic B layer ($t_{B}$), and the outer solvophilic A layer ($t_{Aout}$) 
* Split of the solvophilic scatterers between inner ($s_{Ain}$) and outer layers
* Dispersity in the core radius dimension ($\sigma_{R}$)
* Background intensity as -log10($bg$)

The structural parameters (core radius and the three layer thicknesses) in the `vesicle` shape are coded in Angstroms, and `q` is given in $A^{-1}$.

Just to reiterate, different shapes will have different sets of shape descriptors and parameters, so the aforementioned list of descriptors and their definitions only applies to the **`vesicle`** shape. It is important for the user to make sure that the shape descriptors and minimum/maximum bounds correspond to the expectation of the chosen shape.


To properly use a shape, a user will need to understand what each value in `shape_params` (the shape descriptors) corresponds to and what each value specified in `minvalu` and `maxvalu` refer to (the dimension parameters GA is going to predict based on the input I(q) profile). These pieces of information, as explained in the previous cell, can also be retrieved by calling the docstring of the `scatterer_generator` class of each shape, or checking the API of the relevant shape. For example, for shape _vesicle_,

In [ ]:
# optional to run
import crease_ga.shapes.vesicle.scatterer_generator as sg
print(sg.scatterer_generator.__doc__)

In [ ]:
m.load_shape(shape='vesicle', 
                 shape_params=[24,54,0.5,50.4,50.4,0.55,7], #num_scatterers,N,rho_B,lmono_b,lmono_a,fb,n_LP 
                 minvalu = (50, 30, 30, 30, 0.1, 0.0, 0.1), #lower bounds: R_core,t_Ain,t_B,t_Aout,s_Ain,sigma_R,-log10(bg) 
                 maxvalu = (400, 200, 200, 200, 0.45, 0.45, 4)) # upper bounds: R_core,t_Ain,t_B,t_Aout,s_Ain,sigma_R,-log10(bg)

The final preparation step we need to do before the model is prepared for the GA iteration/optimization is to specify the \"input\" $I_{exp}(q)$, i.e. the input I(q) profile we want to analyze to obtain relevant structural dimensions.

### load_iq

<img src="tutorial_images/Slide5.PNG" width=600 height=600 />
After setting the shape we will be using, we read in the input "experimental" scattering intensity profile $I_{exp}(q)$, which in this example is stored in a text file that has the $q$ and $I(q)$ information each as a column. 

For the example here, our $I_{exp}(q)$ is an *in silico* $I(q)$ created from a target vesicle structure where all dimensions and dispersity in size are known. Thus, the GA can be tested and validated to see how well the GA performs in determining each parameter.

After loading in the target $I_{exp}(q)$, the user can set the upper and lower bounds on the $q$-range over which the GA will determine the best candidate vesicle structure. 

The optional cell block below provides detailed information on the code implementation to load the input $I_{exp}(q)$.

In [ ]:
# optional to run
print(getsource(cga.Model.load_iq))

And now let's load the target $I_{exp}(q)$ for this tutorial and have a look at what we just loaded, the q (`qrange`) and the I(q) (`IQin`):

In [ ]:
m.load_iq('../IEXP_DATA/Itot_disper_10_Ain12_B6_Aout12_nLP7_dR0.2.txt')
import matplotlib.pyplot as plt
plt.loglog(m.qrange,m.IQin)
plt.xlabel('q ($A^{-1}$)')
plt.ylabel('I(q)')
oldq = m.qrange
oldIQ = m.IQin

Sometimes, a user might only be interested in fitting to a portion of the q range. This can be done by setting the `q_bounds` argument of `load_iq` to `[lower_bound,higher_bound]`. For example:

In [ ]:
m.load_iq('../IEXP_DATA/Itot_disper_10_Ain12_B6_Aout12_nLP7_dR0.2.txt',
          q_bounds=[0.005,0.1])
import matplotlib.pyplot as plt
norm = oldIQ[-2]/m.IQin[-1]
plt.loglog(oldq,oldIQ/norm)
plt.loglog(m.qrange,m.IQin)
plt.xlabel('q ($A^{-1}$)')
plt.ylabel('I(q)')

You can see that the first two and last one pairs of q and I(q) values have been excluded because they are out of the bounds that we have set.

## Solve

<img src="tutorial_images/Slide6.PNG" width=600 height=600 />

After defining the input parameters and choosing the shape for the structure, we can start running the GA. This is done by calling the `solve` instance method of the `Model` class. If you are interested, run the block below to see the source code.

In [ ]:
# optional to run
print(getsource(cga.Model.solve))

### Initial_pop

<img src="tutorial_images/Slide7.PNG" width=600 height=600 />

The first thing `solve` does is to call the instance method `initial_pop` to create the first generation of individuals for the GA iteration. `Initial_pop` takes in the number of individuals in a population and generates individual candidate structures based on the bounds set for each of the GA parameters. It then outputs the population information (`pop`) back to the GA code.

To see the source code of this function, run following block:

In [ ]:
# optional to run
print(getsource(cga.utils.initial_pop))

To read the docstring of this function, run following block:

In [ ]:
# optional to run
print(cga.utils.initial_pop.__doc__)

Run the block below to see what a binary representation for an example of 5 individuals in the population looks like:

In [ ]:
# optional to run
pop = cga.utils.initial_pop(popnumber = 5, nloci = 7, numvars = 7)
print(pop)

Starting with the initial population of individuals (0-th generation) generated from `Initial_pop`, we need to determine the "fitness" for each individual, that is, how well does the individual's computed $I_{comp}$(q) match the target "experimental" $I_{exp}$(q). To facilitate the fitness calculation in the GA code, we call the `fitness` function from the main GA code. The fitness calculation requires several input arguments from the main GA code that is passed in binary form, which it then "decodes" to get the values of all of the GA parameters for each of the individuals in the population. 

So before talking about the `fitness` in more detail, first let's take a look at the `decode` function.

### decode

<img src="code_diagram_main9_decode.png" width=600 height=600 />

Run the cell block below to see the source code of this function.

In [ ]:
# optional to run
print(getsource(cga.utils.decode))

To see an example of the decoded parameter values for an individual generated using `initial_pop`, run the following cell block:

In [ ]:
# optional to run
pop = cga.utils.initial_pop(popnumber = 5, nloci = 7, numvars = 7)
print('binary',pop[0])
print('decimal',cga.utils.decode(pop,0,
                                 nloci=7,
                                 minvalu=[50, 30, 30, 30, 0.1, 0.0, 0.1],
                                 maxvalu=[400, 200, 200, 200, 0.45, 0.45, 4]))

Now let's take a closer look at what happens in `fitness` and how fitness is calculated for each individual in the population.
### fitness

<img src="code_diagram_main7_fitness.png" width=600 height=600 />

If you are interested, you can look at the source code:

In [ ]:
# optional to run
print(getsource(cga.Model.fitness))





The core of the `fitness` function is given by the following set of equations

$$ fitness = X(sse_{max}-sse)+Y $$

$$ sse = \sum w\left[log\left(\frac{I_{exp}(q)}{I_{comp}(q)}\right)\right]^2 $$

$$ X = (cs-1)\frac{max(sse_{max}-sse)}{max(sse_{max}-sse)-average(sse_{max}-sse)} $$

$$ Y = (1-X)average(sse_{max}-sse) $$

where $I_{exp}$(q) is the input scattering intensity profile read in from the main GA code, $I_{comp}$(q) is the computed scattering intensity profile for the individual (to be discussed in more details later on), $sse$ is the sum of squared errors between $I_{comp}$(q) and $I_{exp}$(q), and $X$ and $Y$ serve to prevent low fitness solutions from being prematurely eliminated in the population.

In this example, we choose the form of the error function, `sse`, as the sum of the squared log difference of errors where the weighting factor, $w$, accounts for the log-scale spacing of the q values. For more details on the choice of this form for the error function for the systems explored with CREASE, see references [1-5].

We also note that the user may choose to use a different form of sse as long as the choice of error function ensures appropriate error weighting in the range of q explored. For studies where the target $I_{exp}$(q) comes from experimental scattering with an associated uncertainty at each q, a commonly used formulation is $\chi^{2}$ to account for the errors during fitting optimization. For example, SASVIEW uses $\chi^{2}$ to quantify the goodness of fit by taking the squared direct difference between the computed and target dataset over the fitted q range weighted by the squared error at each q. If the error associated with each q is known, $\chi^{2}$ can similarly be implemented into the fitness function in CREASE to account for uncertainty in I(q) at each q. We plan to incorporate $\chi^{2}$ as another option for the fitness calculation in a future release coming soon.

SASVIEW implementation of $\chi^{2}$: https://www.sasview.org/docs/user/qtgui/Perspectives/Fitting/residuals_help.html

## converttoIQ

<img src="code_diagram_main10_convert.png" width=600 height=600 />

In order to perform the fitness calculation, the key piece of information needed is the computed scattering intensity, $I_{comp}$(q), which the `fitness` function gets from the `converttoIQ` function. The `converttoIQ` function is what ties the different shapes to the GA code during implementation. Each shape will have its own `converttoIQ` that takes as input a list of q values (`qrange`) and a list of shape-specific parameters (the "gene"), and outputs a list of $I_{comp}$(q) value corresponding to the input `qrange`. So let's now take a look at how the `vesicle` shape determines the the computed scattering intensity, $I_{comp}$(q), from the input parameters.

### converttoIQ for vesicle

<img src="code_diagram_sub1_all.png" width=600 height=600 />

**Equations to get computed scattering intensity**

The `converttoIQ` function calculates the computed scattering intensity $I_{comp}$(q) from the intra-vesicle structure factor $\omega$(q) (see `LPOmega`) using the following set of equations from scatterers placed within the boundaries of the individual structure:

$$ I_{comp}(q) = F_{M}^2(q)S_{MM}(q) + I_{background} $$

$F^2_{M}(q)$ is the form factor given by
$$ F^2_{M}(q) = \sum_{\alpha\in{A,B}} \sum_{\beta\in{A,B}} b_{\alpha}b_{\beta}F_{GA, \alpha}(q)F_{GA, \beta}(q)\omega(q) $$

where $b_{A}$ and $b_{B}$ are the scattering length densities for A and B scatterers, $F_{GA, \alpha}(q)$ and $F_{GA, \beta}$ are the spherical form factor amplitudes for A and B scatterers (see `LPFbead`), $\omega(q)$ is the intra-vesicle structure factor (see `LPOmega`) and $S_{MM}(q) = 1$ is the vesicle-vesicle structure factor for dilute solutions. For concentration solutions where the structure factor will not be equal 1, we can combine the approach described along with the structure factor CREASE code (work in Reference [4]); we will be adding this capability into this package in the near future.


**Scatterer placements**

As mentioned in an above section, the essential information for the calculation of $I_{comp}$(q) is the scatterer placements, and the user can directly choose the total number of scatterers ($n_{tot}$) to be placed in the vesicle to control the scatterer density. For an assembled structure formed from molecules with two (or more) chemistries, the user also inputs the relative compositions for the chemical groups as $f_{a}$ and $f_{b}$ ($f_{c}$, etc.). The choice of the scatterer density appropriate to the assembled structure studied is chosen by the user. One way to obtain an estimate of $n_{tot}$, as shown in this tutorial example for vesicles, is through the molecular information of the molecule that make up the assembled structure.

In this vesicle system example, let us determine the number of scatterers for placing within each layer in the vesicle wall using the information from the shape parameters section:

1. Get the core radius ($R_{core}$) as the averaged core radius ($R_{core, 0}$) plus the variation in core radius for each replicate ($\sigma_{R}$).
$$ R_{core} = R_{core, 0} + \sigma_{R} $$

2. Calculate the volume of the solvophobic B layer for the individual:
$$ V_{B} = \frac{4}{3}\pi \left[ (R_{core}+t_{Ain}+t_{B})^3 - (R_{core}+t_{Ain})^3 \right] $$

3. Get the number of chains/molecules in the structure:
$$ N_{chain} = \frac{\rho_{B}V_{B}}{Nf_{B}M_{B}} $$

4. The number of A ($n_{A}$) and B ($n_{B}$) scatterers are determined using the composition ($f_{A}$, $f_{B}$) of the respective block in the chain/molecule given by
$$ n_{A} = N_{chain}n_{sct}f_{A} = N_{chain}n_{sct}(1-f_{B}) $$
$$ n_{B} = N_{chain}n_{sct}f_{B} $$
where $n_{sct}$ is the number of scatterers per chain/molecule selected by the user.

5. The numbers of A scatterers in the inner ($n_{Ain}$) and outer ($n_{Aout}$) solvophilic layers are given by
$$ n_{Ain} = n_{A}s_{Ain} $$
$$ n_{Aout} = n_{A}(1-s_{Ain}) $$
where $s_{Ain}$ is a parameter in the GA that gives the fraction of chains/molecules in the inner solvophilic layer. 

**The $s_{Ain}$ parameter, which is determined by the GA, is a feature of the vesicle assembly that cannot be readily determined through traditional fittings of scattering profiles with a core-multi-shell model, which can be found in the [SASVIEW package]( https://www.sasview.org/docs/user/models/core_multi_shell.html). The addition and determination of $s_{Ain}$ in CREASE for vesicles offers molecular insight to the distribution of solvophilic monomers between the inner and outer layers in the vesicle wall**. For more details on the `vesicle` shape scatterer generation and parameter determination, please read reference [5].

Now using the set of information for the vesicle dimensions and the number of scatterers that goes into each layer in the vesicle wall, we can generate the scatterer positions for the vesicle shape by calling the `genLP` and `gen_layer` functions.

### genLP and gen_layer

<img src="code_diagram_sub2_gen.png" width=600 height=600 />

In this example, we get the coordinates for all scatterers in the vesicle with three layers in the wall. 

For the vesicle system example presented here, the `gen_layer` function defined in `gen_layer` places $n_{size}$ number of random point scatterers within a spherical shell defined by the inner ($r_{in}$) and outer ($r_{out}$) radii, where $n_{size}$ is the corresponding number of A or B type scatterers determined in `converttoIQ` for the given shell or layer.

`gen_LP` then calls the `gen_layer` function for each individual layers in the vesicle (in this example, the two solvophilic A and one solvophobic B layers that make up the vesicle membrane) and gathers the scatterer coordinates for the complete vesicle to be passed back to `converttoIQ` for use in calculating the intra-vesicle structure factor $\omega(q)$.

In [ ]:
# optional to run
import crease_ga.shapes.vesicle.scatterer_generator as sg
print(getsource(sg.gen_layer))
print(getsource(sg.genLP))

We have a little visualization method written up for the vesicle shape to help us see how `genLP` works. Let's generate a scatterer configuration based on a random set of dimensions, and see how the scatterers are arranged in 3D. The visualization is interactive, so the user can zoom in and out as well as rotate the vesicle by dragging with their computer mouse.

In [ ]:
# optional to run (but cool visualization)
coords = sg.genLP(Rcore=15,dR_Ain=12,dR_B=8,dR_Aout=12,sigmabead=2,nAin=200,nAout=200,nB=100)
sg.visualize(coords,Rcore=15,dR_Ain=12,dR_B=8,dR_Aout=12,sigmabead=2)

### converttoIQ for vesicle

**Back to getting the computed $I_{comp}$(q)**

Now that we have the scatterer positions from the `gen_LP` function, `converttoIQ` calls the `LPOmega` function and passes the scatterer coordinates to it to get the intra-vesicle structure factor, $\omega(q)$. Recall the set of equations to get $I_{comp}$(q):

$$ I_{comp}(q) = F_{M}^2(q)S_{MM}(q) + I_{background} $$

$F^2_{M}(q)$ is the form factor given by
$$ F^2_{M}(q) = \sum_{\alpha\in{A,B}} \sum_{\beta\in{A,B}} b_{\alpha}b_{\beta}F_{GA, \alpha}(q)F_{GA, \beta}(q)\omega(q) $$

So let us take a look at how $\omega(q)$ is calculated in `LPOmega` and how the spherical form factor amplitudes $F_{GA, \alpha}(q)$ and $F_{GA, \beta}$ for A and B scatterers are calculated in `LPFbead` (note that in this example, A and B scatterers are spheres with the same finite size, so we only need to calculate one spherical form factor as $F_{GA, \alpha}(q)$ = $F_{GA, \beta}(q)$.

### LPOmega

<img src="code_diagram_sub3_omega.png" width=600 height=600 />

Here we calculate the intra-vesicle structure factor as
$$ \omega(q) = \left< \frac{1}{N_{A}+N_{B}} \sum^{N_{A}+N_{B}}_{i=1} \sum^{N_{A}+N_{B}}_{j=1} \frac{\sin{qr_{ij}}}{qr_{ij}} \right> $$

where $N_{A}$ and $N_{B}$ are the number of scatterers in the two solvophilic A and one solvophobic B layers, respectively, and $r_{ij}$ is the distance between the pair of scatterers $i$ and $j$ in the vesicle assembly determined from the previous step in `genLP`.

**NOTE: The above is a computationally intensive calculation as it scales quadratically with the number of scatterers.**

There are alternative faster options for directly obtaining $I_{comp}(q)$ for a given set of structural dimensions include using machine learning methods. We direct you to reference [3] where one such alternative option using artificial neural networks was developed.

To see the sourcecode for this function, run the cell block below:

In [ ]:
# optional to run
print(getsource(sg.LPOmega))

### LPFbead

<img src="code_diagram_sub4_bead.png" width=600 height=600 />

The **spherical form factor amplitude** is calculated as

$$ F_{GA, \alpha}(q) = 3 \frac{ \sin(q0.5\sigma_{GA, \alpha}) - q0.5\sigma_{GA, \alpha}\cos(q0.5\sigma_{GA, \alpha}) }{(q0.5\sigma_{GA, \alpha})^3} $$

where $\alpha$ indicates A or B scatterer and $\sigma_{GA, \alpha}$ is the scatterer diameter.

To see the sourcecode for this function, run the following cell block:

In [ ]:
# optional to run
print(getsource(sg.LPFbead))

### converttoIQ for vesicles

<img src="tutorial_images/Slide10.PNG" width=600 height=600 />

**Back again to getting the computed scattering intensity $I_{comp}$(q)**

Now that we have both $\omega(q)$ and $F_{GA, \alpha}(q)$ from the `LPOmega` and `LPFbead` functions, we put all of the pieces together to get $I_{comp}$(q) as:

$$ I_{comp}(q) = F_{M}^2(q)S_{MM}(q) + I_{background} $$

$F^2_{M}(q)$ is the form factor given by
$$ F^2_{M}(q) = \sum_{\alpha\in{A,B}} \sum_{\beta\in{A,B}} b_{\alpha}b_{\beta}F_{GA, \alpha}(q)F_{GA, \beta}(q)\omega(q) $$

and `converttoIQ` will output the computed scattering intensity $I_{comp}$(q) as `IQid` in the code.

Run the cell block below to see the sourcecode for this function:

In [ ]:
# optional to run
print(getsource(sg.scatterer_generator.converttoIQ))

### fitness

<img src="tutorial_images/Slide8.PNG" width=600 height=600 />

After computing the scattering intensity, we are finally back to the `fitness` function, which if the user recalls is given by the following set of equations

$$ fitness = X(sse_{max}-see)+Y $$

$$ sse = \sum w\left[log\left(\frac{I_{exp}(q)}{I_{comp}(q)}\right)\right]^2 $$

$$ X = (cs-1)\frac{max(sse_{max}-sse)}{max(sse_{max}-sse)-average(sse_{max}-sse)} $$

$$ Y = (1-X)average(sse_{max}-sse) $$

where we get the $I_{exp}$(q) from reading in the input scattering data file and $I_{comp}$(q) from the `converttoIQ` function, and in this example we choose the scaling constant $cs$ to be 10.

Thus the `fitness` function will calculate the fitness value for each of the individuals. Also within the `fitness` function, we calculate the selection probability for each individual, $P_{select}$, that is proportional to its fitness value.
We then store the selection information (`pacc`) for all of the individuals in the population and pass it back to the main GA code where individuals will be selected to move on to the next generation.

To see the source code of this function, run the following cell block:

In [ ]:
# optional to run
print(getsource(cga.Model.fitness))

In this example, the `fitness` function also provides useful information like which one of the individuals performed the best ("elite") in terms of having its $I_{comp}$(q) most closely match the input $I_{exp}$(q), the second best ("second"), the average ("average"), and the worst ("minfit") that are stored and saved for the user (in `fitness_vs_gen.txt`).

Next, a new generation of individuals will be selected based on the fitnesses of the current generation. That is done in the `genetic_operations` function.

### genetic_operations

<img src="tutorial_images/Slide9.PNG" width=600 height=600 />

**Selection Process: Crossover, Mutation, and Elitism**

Now that we have the fitness value for each individual in the population, the GA will need to select the candidates for the next generation. Where the next generation has its fitness calculated, and this iterative process is continued until the maximum number of generations (set by the user during the beginning of the main GA code) is reached. As each of the GA parameters is encoded as a seven-digit binary sequence, the genetic operations (crossover, mutation) performed on selected individuals is done in binary form.

*Crossover*

During crossover, two individuals in the population are selected as "parents" and their traits are mixed to produced a new "offspring" individual to be included in the next generation. Each "parent" individual is selected with probability $PC$, and the offspring has traits randomly chosen from its two "parent" individuals. 

*Mutation*

An individual is selected for mutation with probability $PM$, where one of its traits is randomly changed within that parameter's range of values (between the min value and max value).

*Elitism*

The best individual in the generation is always included in the population for next generation.

Run the block below to see the source code:

In [ ]:
# optional to run
print(getsource(cga.Model.genetic_operations))

## Adapting the genetic operation parameters during GA
The probabilities of crossover $PC$ and mutation $PM$ are adjusted over the generations during a GA run to help prevent premature convergence to local minima (with the `adaptation_params.update` function during `Model.solve`). These self-adjustments are controlled by a set of adaptation parameters. During the GA, $PC$ and $PM$ are updated after each generation based on the reciprocal genetic diversity parameter, $GDM$, given by

$$ GDM = \frac{\text{min error of population}}{\text{average error of population}} $$

which also has upper and lower bounds ($GDM_{max}$, $GDM_{min}$) set by the user. $GDM$ should be between 0 and 1, with higher value indicating lower diversity in the generation. When $GDM$ becomes smaller than $GDM_{min}$, $PM$ is automatically decreased (by a factor of $kGDM$) and $PC$ is increased (by a factor of $kGDM$) to encourage more crossover, and thus decrease diversity (higher $GDM$). When $GDM$ becomes larger than $GDM_{max}$, $PM$ is increased (by a factor of $kGDM$) and $PC$ is decreased (by a factor of $kGDM$) to encourage mutation and increase diversity. The adjustment factor $kGDM$ is also set by the user and needs to be larger than 1.

These values can depend on the shape and need to be tested if the user applies the GA to a new shape. For this tutorial, we are using the adaptation parameters that have been optimized for the vesicle example here. To learn more about the choosing these adaptation parameters, please read referefence [1].

Run the cell block below to see the values of the adaptation parameters being used:

In [ ]:
# optional to run
for d in dir(m.adaptation_params):
    if not '__' in d:
    
        print(d,getattr(m.adaptation_params,d))

After selecting the individuals in the population that move on to the next generation, we once again calculate the fitness for each individual in the new generation. Then the selection and fitness processes are repeated until the end of the GA run.

## Running the GA

We have finally gone through all the elements of `Model.solve`, and now let's get it to work!
(**Note**: you can specify the `output_dir` argument for the directory to which you would like to save all the output files. The directory needs to already exist before you run this line!)

In [ ]:
'''uncomment this to create a new directory'''
#!mkdir test_output_1 

m.solve(verbose=True,output_dir='./test_output_1/')

If all of the required cell blocks of code above have been run properly, then performing the GA on the small number of individuals and generations will produce the output dimensions and $I_{comp}$(q) that can be compared to the target dimensions of the structure that provided the input $I_{exp}$(q) as shown below.

Data output from each generation has been saved to files under the directory specified by `output_dir`. Check out `results` for all the indiviuals at each generation, and `fitness_vs_gen` for how the fitness of the generation in general evolves. 

After you are finished, check out the notebook under this directory named [workflow_simplified](./workflow_simplified.ipynb), to see how each step works once we have taken out these in-depth explanations. You will see how the entire GA is actually set up and run with few lines of codes.

Good luck!

<img src="short_run_output.png" width=300 height=300 />

### GA for Vesicles with Larger Population Size and Number of Generations

If the above codes for the example vesicle system in this tutorial have been successfully run, then the user should be able to see the best individual's $I_{comp}$(q) plotted against the input $I_{exp}$(q). With the small population size (`popnumber = 5`) and number of generations (`generations = 5`) used in this tutorial example, the resulting $I_{comp}$(q) of the best individual structure is unlikely to be a good match to the input $I_{exp}$(q). As mentioned previously, using a small number of individuals and generations in this example is to purely to keep the calculations computationally feasible for a tutorial. To produce better matches to the input $I_{exp}$(q) one would need to use significantly higher number of individuals and generations (and possibly a more powerful computer to speed the GA up).

Shown below is an example of GA determined results that we can expect when we use the GA on the same input $I_{exp}$(q) but with a much larger population size and number of generations:

`popnumber = 80`

`generations = 100`

<img src="crease_tutorial_fig_Icomp-vs-gen.png" width=300 height=300 />

**Figure 1**: *CREASE analysis on an example vesicle with the target dimensions of small core with thin B layer (shown schematically). $I_{comp}$(q) of the best individual (colored lines) from all generations in one example GA run (with dispersity) compared with the input in silico  $I_{exp}$(q) (open circles). Progression from the initial generation to the final generation is shown with increasingly warmer colors.*

From the best individual's $I_{comp}$(q) over multiple generations, we can see that increasing the population size and number of generation gradually drives the GA determined vesicle structure toward that of the target structure (**Figure 1**). Because our input  $I_{exp}$(q) was generated in silico from known target dimensions, we can also make a direct comparison of the GA determined vesicle dimensions to the known target dimensions. For comparison we also provide the dimensions determined through the traditional fitting of the data with the core-multi-shell model in SASVIEW.

<img src="crease_tutorial_fig_dim-ga-vs-sasview.png" width=300 height=300 />

**Figure 2**: *GA determined dimensions of relevant vesicle features at the end of 100 generations for the cases where dispersity in dimensions is not accounted for in the GA step (blue triangles) and when dispersity is incorporated into the GA (orange squares). The target dimensions of the in silico Iexp(q) are shown as open circles with 20% dispersity in the core radius (Rcore) dimensions. SASVIEW fitting results using the core-multi-shell model (grey stars) are offset to the right of the target dimensions.*

As discussed during the tutorial, GA for vesicles can incorporate the effect of dispersity in the dimensions of the assembly into the calculation of the individual's $I_{comp}$(q). This dispersity is determined as one of the GA results given as $\sigma_{R}$ in this example systems where dispersity is incorporated in the core radius dimension. **Figure 2** shows how the GA with (i.e. GA searches and optimizes for $\sigma_{R}$) and without (i.e. GA sets $\sigma_{R} = 0$) dispersity performs in getting the vesicle structural dimensions of core radius ($R_{core}$), each layer thickness ($t_{A,in}$, $t_{B}$, $t_{A,out}$) and total vesicle diameter size ($D_{vesicle}$). We can see that with an appropriate population size and number of generations, the GA is able to accurately capture the target structure's dimensions. Additionally, while the analytical fitting to a core-multi-shell model using SASVIEW can also determine these structural dimensions, GA for vesicle is able to inform the user how the molecules are split between the inner and outer layers of the vesicle wall ($s_{Ain}$), a piece of molecular information not available if the core-multi-shell model is used instead. The GA determined parameter $s_{Ain}$ gives the fraction of molecules in the vesicle assembly that occupies the inner layer of the vesicle walls. For this example system, the target structure has 20% of the molecules in the assembly occupying the inner layer of the vesicle wall and the GA is able to determine $s_{Ain}$ as 28%, matching the target value.

For more detailed discussions on CREASE for vesicles and its application to different vesicle sizes and high dispersity, the user can refer to reference [5].